In [1]:
import sys
sys.path.append('..')
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from sympy import simplify_logic
import time
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.tree import _tree, export_text

from deep_logic.utils.base import validate_network, set_seed, tree_to_formula
from deep_logic.utils.relu_nn import get_reduced_model, prune_features
from deep_logic.utils.psi_nn import prune_equal_fanin
from deep_logic import logic

results_dir = 'results/xor'
if not os.path.isdir(results_dir):
    os.makedirs(results_dir)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
concepts = ['c1', 'c2', 'c3', 'c4']
n_rep = 10
tot_epochs = 4001
prune_epochs = 2000

In [2]:
# XOR problem
x_train = torch.tensor([
    [0, 0, 0, 1],
    [0, 1, 0, 1],
    [1, 0, 0, 1],
    [1, 1, 0, 1],
], dtype=torch.float)
y_train = torch.tensor([0, 1, 1, 0], dtype=torch.long)


x_test = torch.tensor([
    [0, 0.95, 0, 1],
    [0, 0.9, 0.2, 0.7],
    [0.05, 1, 0.5, 1],
    [0.1, 0.8, 0.1, 0.8],
    [0.45, 1, 0.95, 0.9],
    [0, 0.35, 0.95, 0.9],
    [0.95, 0.9, 0.75, 0.2],
    [0.75, 0.2, 0.75, 0.15],
    [0.75, 0.15, 0.75, 0.2],
], dtype=torch.float)
y_test = torch.tensor([1, 1, 1, 1, 1, 0, 0, 1, 1], dtype=torch.long)

In [3]:
def train_nn(x_train, y_train, need_pruning, seed, device, relu=False, verbose=False):
    set_seed(seed)
    x_train = x_train.to(device)
    y_train = y_train.to(device)
    layers = [
        torch.nn.Linear(x_train.size(1), 10),
        torch.nn.ReLU() if relu else torch.nn.LeakyReLU(),
        torch.nn.Linear(10, 4),
        torch.nn.ReLU() if relu else torch.nn.LeakyReLU(),
        torch.nn.Linear(4, 2),
        torch.nn.Softmax(dim=1),
    ]
    model = torch.nn.Sequential(*layers).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_form = torch.nn.CrossEntropyLoss()
    model.train()
    for epoch in range(tot_epochs):
        # forward pass
        optimizer.zero_grad()
        y_pred = model(x_train)
        # Compute Loss
        loss = loss_form(y_pred, y_train)

        for module in model.children():
            if isinstance(module, torch.nn.Linear):
                loss += 0.004 * torch.norm(module.weight, 1)
                loss += 0.004 * torch.norm(module.bias, 1)
                break

        # backward pass
        loss.backward()
        optimizer.step()

        if epoch > prune_epochs and need_pruning:
            prune_features(model, n_classes=1, device=device)
            need_pruning = False
            
        # compute accuracy
        if epoch % 500 == 0 and verbose:
            y_pred_d = torch.argmax(y_pred, dim=1)
            accuracy = y_pred_d.eq(y_train).sum().item() / y_train.size(0)
            print(f'\t Epoch {epoch}: train accuracy: {accuracy:.4f}')
    
    return model

In [7]:
def train_psi_nn(x_train, y_train, need_pruning, seed, device, verbose=False):
    set_seed(seed)
    x_train = x_train.to(device)
    y_train = y_train.to(device).to(torch.float)
    layers = [
        torch.nn.Linear(x_train.size(1), 10),
        torch.nn.Sigmoid(),
        torch.nn.Linear(10, 4),
        torch.nn.Sigmoid(),
        torch.nn.Linear(4, 1),
        torch.nn.Sigmoid(),
    ]
    model = torch.nn.Sequential(*layers).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_form = torch.nn.BCELoss()
    model.train()
    for epoch in range(tot_epochs):
        # forward pass
        optimizer.zero_grad()
        y_pred = model(x_train).squeeze()
        # Compute Loss
        loss = loss_form(y_pred, y_train)

        for module in model.children():
            if isinstance(module, torch.nn.Linear):
                loss += 0.00001 * torch.norm(module.weight, 1)

        # backward pass
        loss.backward()
        optimizer.step()

        if epoch > prune_epochs and need_pruning:
            model = prune_equal_fanin(model, 2, validate=True, device=device)
            need_pruning = False
            
        # compute accuracy
        if epoch % 500 == 0 and verbose:
            y_pred_d = y_pred > 0.5
            accuracy = y_pred_d.eq(y_train).sum().item() / y_train.size(0)
            print(f'\t Epoch {epoch}: train accuracy: {accuracy:.4f}')
    
    return model

In [8]:
def c_to_y(method, need_pruning, relu, verbose=False):
    methods = []
    splits = []
    explanations = []
    explanations_inv = []
    model_accuracies = []
    explanation_accuracies = []
    explanation_accuracies_inv = []
    elapsed_times = []
    elapsed_times_inv = []
    for seed in range(n_rep):
        explanation, explanation_inv = '', ''
        explanation_accuracy, explanation_accuracy_inv = 0, 0
        
        print(f'Seed [{seed+1}/{n_rep}]')
        
        if method == 'tree':
            classifier = DecisionTreeClassifier(random_state=seed)
            classifier.fit(x_train.detach().numpy(), y_train.detach().numpy())
            y_preds = classifier.predict(x_test.detach().numpy())
            model_accuracy = accuracy_score(y_test.detach().numpy(), y_preds)

            target_class = 1
            start = time.time()
            explanation = tree_to_formula(classifier, concepts, target_class)
            elapsed_time = time.time() - start

            target_class_inv = 0
            start = time.time()
            explanation_inv = tree_to_formula(classifier, concepts, target_class_inv)
            elapsed_time = time.time() - start
        
        else:
            if method == 'psi':
                # positive class
                target_class = 1
                model = train_psi_nn(x_train, y_train.eq(target_class), need_pruning, seed, device, verbose)
                y_preds = model(x_test.to(device)).cpu().detach().numpy()
                model_accuracy = accuracy_score(y_test.cpu().detach().numpy(), y_preds>0.5)
                
            else:
                model = train_nn(x_train, y_train, need_pruning, seed, device, relu, verbose)
                y_preds = model(x_test.to(device)).cpu().detach().numpy()
                model_accuracy = accuracy_score(y_test.cpu().detach().numpy(), y_preds.argmax(axis=1))

            # positive class
            target_class = 1
            start = time.time()
            if method == 'psi':
                global_explanation = logic.generate_fol_explanations(model, device)[0]
            else:
                global_explanation, _, _ = logic.relu_nn.combine_local_explanations(model, 
                                                                                   x_train.to(device), y_train.to(device), 
                                                                                   target_class=target_class,
                                                                                   method=method, device=device)
            elapsed_time = time.time() - start
            
            if global_explanation:
                explanation_accuracy, _ = logic.base.test_explanation(global_explanation, target_class, x_test, y_test)
                explanation = logic.base.replace_names(global_explanation, concepts)

            # negative class
            target_class_inv = 0
            if method == 'psi':
                model = train_psi_nn(x_train, y_train.eq(target_class_inv), need_pruning, seed, device, verbose)
            
            start = time.time()
            if method == 'psi':
                global_explanation_inv = logic.generate_fol_explanations(model, device)[0]
            else:
                global_explanation_inv, _, _ = logic.relu_nn.combine_local_explanations(model, 
                                                                                       x_train.to(device), 
                                                                                       y_train.to(device), 
                                                                                       target_class=target_class_inv,
                                                                                       method=method, device=device)
            elapsed_time_inv = time.time() - start
            if global_explanation_inv:
                explanation_accuracy_inv, _ = logic.base.test_explanation(global_explanation_inv, 
                                                                          target_class_inv, x_test, y_test)
                explanation_inv = logic.base.replace_names(global_explanation_inv, concepts)
        
        if verbose:
            print(f'\t Model\'s accuracy: {model_accuracy:.4f}')
            print(f'\t Class {target_class} - Global explanation: "{explanation}" - Accuracy: {explanation_accuracy:.4f}')
            print(f'\t Elapsed time {elapsed_time}')
            print(f'\t Class {target_class} - Global explanation: "{explanation_inv}" - Accuracy: {explanation_accuracy_inv:.4f}')
            print(f'\t Elapsed time {elapsed_time_inv}')

        methods.append(method)
        splits.append(seed)
        explanations.append(explanation)
        explanations_inv.append(explanation_inv)
        model_accuracies.append(model_accuracy)
        explanation_accuracies.append(explanation_accuracy)
        explanation_accuracies_inv.append(explanation_accuracy_inv)
        elapsed_times.append(elapsed_time)
        elapsed_times_inv.append(elapsed_time_inv)
    
    results = pd.DataFrame({
        'method': methods,
        'split': splits,
        'explanation': explanations,
        'explanation_inv': explanations_inv,
        'model_accuracy': model_accuracies,
        'explanation_accuracy': explanation_accuracies,
        'explanation_accuracy_inv': explanation_accuracies_inv,
        'elapsed_time': elapsed_times,
        'elapsed_time_inv': elapsed_times_inv,
    })
    results.to_csv(os.path.join(results_dir, f'results_{method}.csv'))
    
    return results

# General pruning

In [9]:
method = 'pruning'
need_pruning = True
relu = False
results_pruning = c_to_y(method, need_pruning, relu)
results_pruning

Seed [1/10]
Seed [2/10]
Seed [3/10]
Seed [4/10]
Seed [5/10]
Seed [6/10]
Seed [7/10]
Seed [8/10]
Seed [9/10]
Seed [10/10]


,method,split,explanation,explanation_inv,model_accuracy,explanation_accuracy,explanation_accuracy_inv,elapsed_time,elapsed_time_inv
0,pruning,0,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,100.000000,100.000000,0.028214,0.021940
1,pruning,1,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,100.000000,100.000000,0.021931,0.021910
2,pruning,2,(c1 & c4 & ~c2) | (c2 & c4 & ~c1),(c1 & c2 & c4) | (c4 & ~c1 & ~c2),1.0,77.777778,88.888889,0.030917,0.025931
3,pruning,3,(c1 & c4 & ~c2) | (c2 & c4 & ~c1),(c1 & c2 & c4) | (c4 & ~c1 & ~c2),1.0,77.777778,88.888889,0.022954,0.021925
4,pruning,4,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,100.000000,100.000000,0.038900,0.031913
5,pruning,5,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,100.000000,100.000000,0.021974,0.017951
6,pruning,6,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,100.000000,100.000000,0.020945,0.017920
7,pruning,7,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,100.000000,100.000000,0.026927,0.023955
8,pruning,8,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,100.000000,100.000000,0.018949,0.016954
9,pruning,9,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,100.000000,100.000000,0.018950,0.016954


# LIME

In [10]:
method = 'lime'
need_pruning = False
relu = False
results_lime = c_to_y(method, need_pruning, relu)
results_lime

Seed [1/10]
Seed [2/10]
Seed [3/10]
Seed [4/10]
Seed [5/10]
Seed [6/10]
Seed [7/10]
Seed [8/10]
Seed [9/10]
Seed [10/10]


,method,split,explanation,explanation_inv,model_accuracy,explanation_accuracy,explanation_accuracy_inv,elapsed_time,elapsed_time_inv
0,lime,0,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,100.0,100.0,0.867717,0.859700
1,lime,1,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,100.0,100.0,0.770936,0.977386
2,lime,2,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,100.0,100.0,0.823947,0.979410
3,lime,3,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,100.0,100.0,0.719045,0.754099
4,lime,4,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,100.0,100.0,0.738190,0.721677
5,lime,5,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,100.0,100.0,0.716081,0.733038
6,lime,6,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,100.0,100.0,0.720073,0.733165
7,lime,7,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,100.0,100.0,0.690149,0.695138
8,lime,8,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,100.0,100.0,0.693132,0.681211
9,lime,9,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,100.0,100.0,0.704674,0.709137


# Relu

In [11]:
method = 'weights'
need_pruning = False
relu = True
results_weights = c_to_y(method, need_pruning, relu)
results_weights

Seed [1/10]
Seed [2/10]
Seed [3/10]
Seed [4/10]
Seed [5/10]
Seed [6/10]
Seed [7/10]
Seed [8/10]
Seed [9/10]
Seed [10/10]


,method,split,explanation,explanation_inv,model_accuracy,explanation_accuracy,explanation_accuracy_inv,elapsed_time,elapsed_time_inv
0,weights,0,c1 | c2,(c1 & c2) | (~c1 & ~c2),0.888889,88.888889,100.000000,0.020977,0.020940
1,weights,1,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,100.000000,100.000000,0.020945,0.020943
2,weights,2,~c1 | ~c2,c1 | ~c2,1.000000,88.888889,77.777778,0.021916,0.021942
3,weights,3,~c1 | ~c2,(c4 & ~c1 & ~c2) | (c1 & c2 & c4 & ~c3),1.000000,88.888889,88.888889,0.017952,0.025931
4,weights,4,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,100.000000,100.000000,0.019947,0.019947
5,weights,5,(c1 & ~c2) | (c2 & c4 & ~c1 & ~c3),(c1 & c2 & c4) | (~c1 & ~c2),1.000000,88.888889,88.888889,0.026909,0.023925
6,weights,6,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,100.000000,100.000000,0.020895,0.021932
7,weights,7,~c1 | ~c2,c2 | ~c1,1.000000,88.888889,44.444444,0.019946,0.015960
8,weights,8,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,100.000000,100.000000,0.019946,0.018949
9,weights,9,(c2 & ~c1) | (c1 & c4 & ~c2 & ~c3),~c2,0.888889,77.777778,66.666667,0.027894,0.017955


# Psi network

In [12]:
method = 'psi'
need_pruning = True
relu = False
results_psi = c_to_y(method, need_pruning, relu, verbose=True)
results_psi

Seed [1/10]
	 Epoch 0: train accuracy: 0.5000
	 Epoch 500: train accuracy: 0.7500
	 Epoch 1000: train accuracy: 0.7500
	 Epoch 1500: train accuracy: 0.7500
	 Epoch 2000: train accuracy: 1.0000
	 Epoch 2500: train accuracy: 0.5000
	 Epoch 3000: train accuracy: 1.0000
	 Epoch 3500: train accuracy: 1.0000
	 Epoch 4000: train accuracy: 1.0000
	 Epoch 0: train accuracy: 0.5000
	 Epoch 500: train accuracy: 0.5000
	 Epoch 1000: train accuracy: 0.5000
	 Epoch 1500: train accuracy: 1.0000
	 Epoch 2000: train accuracy: 1.0000
	 Epoch 2500: train accuracy: 0.7500
	 Epoch 3000: train accuracy: 0.7500
	 Epoch 3500: train accuracy: 0.7500
	 Epoch 4000: train accuracy: 0.7500
	 Model's accuracy: 1.0000
	 Class 1 - Global explanation: "((c1 & ~c2) | (c2 & ~c1))" - Accuracy: 100.0000
	 Elapsed time 0.03089308738708496
	 Class 1 - Global explanation: "(~c1 & ~c2)" - Accuracy: 88.8889
	 Elapsed time 0.043914079666137695
Seed [2/10]
	 Epoch 0: train accuracy: 0.5000
	 Epoch 500: train accuracy: 0.7500
	 E

	 Epoch 500: train accuracy: 0.7500
	 Epoch 1000: train accuracy: 0.7500
	 Epoch 1500: train accuracy: 0.7500
	 Epoch 2000: train accuracy: 0.7500
	 Epoch 2500: train accuracy: 0.7500
	 Epoch 3000: train accuracy: 0.7500
	 Epoch 3500: train accuracy: 0.7500
	 Epoch 4000: train accuracy: 0.7500
	 Epoch 0: train accuracy: 0.5000
	 Epoch 500: train accuracy: 0.5000
	 Epoch 1000: train accuracy: 0.7500
	 Epoch 1500: train accuracy: 0.7500
	 Epoch 2000: train accuracy: 0.7500
	 Epoch 2500: train accuracy: 0.7500
	 Epoch 3000: train accuracy: 0.7500
	 Epoch 3500: train accuracy: 0.7500
	 Epoch 4000: train accuracy: 0.7500
	 Model's accuracy: 0.8889
	 Class 1 - Global explanation: "(c1 | c2)" - Accuracy: 88.8889
	 Elapsed time 0.03095555305480957
	 Class 1 - Global explanation: "(c1 | ~c2)" - Accuracy: 77.7778
	 Elapsed time 0.03391218185424805


,method,split,explanation,explanation_inv,model_accuracy,explanation_accuracy,explanation_accuracy_inv,elapsed_time,elapsed_time_inv
0,psi,0,((c1 & ~c2) | (c2 & ~c1)),(~c1 & ~c2),1.000000,100.000000,88.888889,0.030893,0.043914
1,psi,1,((c1 & ~c2) | (c2 & ~c1)),((c1 & c2) | (~c1 & ~c2)),1.000000,100.000000,100.000000,0.032911,0.041888
2,psi,2,((c1 & ~c2) | (c2 & ~c1)),((c1 & c2) | (~c1 & ~c2)),1.000000,100.000000,100.000000,0.040917,0.047872
3,psi,3,((c1 & ~c2) | (c2 & ~c1)),(c2 | ~c1),1.000000,100.000000,44.444444,0.039920,0.040889
4,psi,4,((c1 & ~c2) | (c2 & ~c1)),((c1 & c2) | (~c1 & ~c2)),1.000000,100.000000,100.000000,0.033910,0.049838
5,psi,5,(c2 & ~c1),((c1 & c2) | (~c1 & ~c2)),0.777778,77.777778,100.000000,0.036901,0.029919
6,psi,6,(c2 & ~c1),(~c1 & ~c2),0.666667,77.777778,88.888889,0.031943,0.032942
7,psi,7,(c1 & ~c2),((c1 & c2) | (~c1 & ~c2)),0.444444,44.444444,100.000000,0.034910,0.035938
8,psi,8,((c1 & ~c2) | (c2 & ~c1)),(c1 & c2),1.000000,100.000000,88.888889,0.040887,0.022938
9,psi,9,(c1 | c2),(c1 | ~c2),0.888889,88.888889,77.777778,0.030956,0.033912


# Decision tree

In [13]:
method = 'decision_tree'
need_pruning = False
relu = False
results_tree = c_to_y(method, need_pruning, relu)
results_tree

Seed [1/10]
Seed [2/10]
Seed [3/10]
Seed [4/10]
Seed [5/10]
Seed [6/10]
Seed [7/10]
Seed [8/10]
Seed [9/10]
Seed [10/10]


,method,split,explanation,explanation_inv,model_accuracy,explanation_accuracy,explanation_accuracy_inv,elapsed_time,elapsed_time_inv
0,decision_tree,0,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,100.000000,100.000000,0.021960,0.022921
1,decision_tree,1,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,100.000000,100.000000,0.020945,0.019965
2,decision_tree,2,(c1 & ~c2) | (c2 & ~c1),(c1 & c2 & c4) | (~c1 & ~c2),1.0,100.000000,88.888889,0.020944,0.022938
3,decision_tree,3,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (c4 & ~c1 & ~c2),1.0,100.000000,100.000000,0.020940,0.023910
4,decision_tree,4,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,100.000000,100.000000,0.021941,0.021945
5,decision_tree,5,c1 | c2,(c1 & c2) | (~c1 & ~c2),1.0,88.888889,100.000000,0.019947,0.019946
6,decision_tree,6,~c1 | ~c2,c2 | ~c1,1.0,88.888889,44.444444,0.021941,0.019947
7,decision_tree,7,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,100.000000,100.000000,0.021941,0.019947
8,decision_tree,8,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,100.000000,100.000000,0.021941,0.019947
9,decision_tree,9,(c1 & c4 & ~c2) | (c2 & c4 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,77.777778,100.000000,0.027918,0.024946


# Summary

In [14]:
cols = ['model_accuracy', 'explanation_accuracy', 'explanation_accuracy_inv', 'elapsed_time', 'elapsed_time_inv']
mean_cols = [f'{c}_mean' for c in cols]
sem_cols = [f'{c}_sem' for c in cols]

# pruning
df_mean = results_pruning[cols].mean()
df_sem = results_pruning[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_pruning = pd.concat([df_mean, df_sem])
summary_pruning.name = 'pruning'

# lime
df_mean = results_lime[cols].mean()
df_sem = results_lime[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_lime = pd.concat([df_mean, df_sem])
summary_lime.name = 'lime'

# weights
df_mean = results_weights[cols].mean()
df_sem = results_weights[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_weights = pd.concat([df_mean, df_sem])
summary_weights.name = 'weights'

# psi
df_mean = results_psi[cols].mean()
df_sem = results_psi[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_psi = pd.concat([df_mean, df_sem])
summary_psi.name = 'psi'

# tree
df_mean = results_tree[cols].mean()
df_sem = results_tree[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_tree = pd.concat([df_mean, df_sem])
summary_tree.name = 'tree'

summary = pd.concat([summary_pruning, summary_lime, summary_weights, summary_psi, summary_tree], axis=1).T
summary.columns = mean_cols + sem_cols
summary

,model_accuracy_mean,explanation_accuracy_mean,explanation_accuracy_inv_mean,elapsed_time_mean,elapsed_time_inv_mean,model_accuracy_sem,explanation_accuracy_sem,explanation_accuracy_inv_sem,elapsed_time_sem,elapsed_time_inv_sem
pruning,1.000000,95.555556,97.777778,0.025066,0.021736,0.000000,2.962963,1.481481,0.001985,0.001494
lime,1.000000,100.000000,100.000000,0.744394,0.784396,0.000000,0.000000,0.000000,0.018698,0.035799
weights,0.977778,92.222222,86.666667,0.021733,0.020842,0.014815,2.371527,5.925926,0.001003,0.000909
psi,0.877778,88.888889,88.888889,0.035415,0.038005,0.060745,5.737753,5.493480,0.001264,0.002656
tree,1.000000,95.555556,93.333333,0.022042,0.021641,0.000000,2.456759,5.543196,0.000688,0.000613


In [15]:
summary.to_csv(os.path.join(results_dir, 'summary.csv'))